In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

_ = load_dotenv(find_dotenv())

chat_model = ChatGroq(model = "llama-3.1-8b-instant")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [3]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def add_russ_last_name(name: str) -> str:
    return name + " ovich"

chain = RunnablePassthrough() | RunnableLambda(add_russ_last_name)

chain.invoke("Russ") 

'Russ ovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [5]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "op-1" : RunnablePassthrough(),
        "op-2" : RunnableLambda(add_russ_last_name)
    }
)

chain.invoke("Russ") 

{'op-1': 'Russ', 'op-2': 'Russ ovich'}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {subject}.")

chain = RunnableParallel(
    {
        "op-1" : RunnablePassthrough(),
        "subject" : RunnableLambda(add_russ_last_name),
        "op-3" : RunnablePassthrough()
    } 
) | prompt | chat_model | StrOutputParser()

chain.invoke("Russ")

'Why did Russ Owenich bring a ladder to the party? \n\nBecause he heard the drinks were on the house.'